# Přednáška 8: Variační Bayesova aproximace, bayesovská lineární regrese

## Obsah přednášky
- pravděpodobnostní modely a souvislost s optimalizací
- variační Bayesova aproximace
    - ukážeme si skalární příklad
- bayesovská lineární regrese
    - porovnáme s dosavadními výsledky

## Bayesovský přístup - postup ve zkratce

- volba modelu dat
- volba apriorního rozdělení parametrů
- výpočet aposteriorních rozdělení parametrů

### Analogie s optimalizací

Analogii si ukážeme na úloze lineární regrese, kde jsme v předchozí přednášce formulovali lineární regresi jako optimalizační problém.

|  | bayesovská formulace | formulace pomocí optimalizace |
| --- | --- | ---
| model dat | $f(y|\beta)=\mathcal{N}(X\beta,I_p)$ | $||(\mathbf{y} - X\beta)^T(\mathbf{y} - X\beta)||_2^2$ |
| apriorno | $f(\beta) = \dots$ | $g(\beta)=\alpha||\beta||_x$ | 


## Variační Bayesova (VB) metoda 

Mějme model dat daný jako $f(\mathbf{y}|\theta)$, kde $\theta$ je $q$-členná množina parametrů, $q>2$. Nechť $\theta_1,\dots,\theta_q$ podmíněně nezávislé proměnné, tedy platí
$$
f\left(\theta_{1},\theta_{2},\dots,\theta_{q}\right)=\prod_{i=1}^{q}f(\theta_{i}).
$$
Potom pro aposteriorní rozdělení $\tilde{f}(\theta_{i}|\mathbf{y})$, $i=1,\dots,q$ platí
$$
\tilde{f}\left(\theta_{i}|\mathbf{y}\right)\propto\exp\left[E_{\tilde{f}(\theta_{\backslash i}|\mathbf{y})}\left(\ln f(\theta_{1},\theta_{2},\dots,\theta_{q},\mathbf{y})\right)\right],
$$
kde symbol $\theta_{\backslash i}$ značí všechny proměnné z dané množiny až na i-tou a symbol $\propto$ značí rovnost až na normovací konstantu.

### Příklad: skalární dekompozice

Mějme model skalární dekompozice 
$$
d = ax + e,
$$
kde $d$ jsou data, $a$ a $x$ jsou neznámé proměnné a $e = \mathcal{N}(0,r_e)$. Model dat tedy můžeme zapsat jako 
$$
f(d|a,x) = \mathcal{N}(ax,r_e).
$$
Apriorní modely pro $a$ a $x$ zvolíme jako
$$
f(a)= \mathcal{N}(0,r_{a}),\,\,\,\,\,\,\,\, f(x)= \mathcal{N}(0,r_{x}).
$$
Data $d$ a parametry $r_e$, $r_a$ a $r_x$ předpokládáme jako známé. Cílem je určit odhady $\widehat{a}$
a $\widehat{x}$.

Sdružená věrohodnost je
$$
f(a,x,d)\propto\exp\left(-\frac{1}{2}\left(\frac{(ax-d)^{2}}{r_{e}}+\frac{a^{2}}{r_{a}}+\frac{x^{2}}{r_{x}}\right)\right),
$$
a my si vyjádříme logaritmus sdružené věrohodnosti (ten přesně pak potřebujeme do VB metody):
$$
\ln f(a,x,d)\propto-\frac{1}{2}\left(\frac{(ax-d)^{2}}{r_{e}}+\frac{a^{2}}{r_{a}}+\frac{x^{2}}{r_{x}}\right),
$$
který dosadíme do rovnice pro aposteriorno:
$$
\tilde{f}\left(a|d\right)\propto \exp\left[E_{\tilde{f}(x|d)}\left(-\frac{1}{2}a^{2}\left(x^{2}r_{e}^{-1}+r_{a}^{-1}\right)-a\left(dxr_{e}^{-1}\right)\right)\right] = \exp\left[-\frac{1}{2}a^{2}\left(\widehat{x^{2}}r_{e}^{-1}+r_{a}^{-1}\right)-a\left(d\widehat{x}r_{e}^{-1}\right)\right],\\
\tilde{f}\left(x|d\right)\propto \exp\left[E_{\tilde{f}(a|d)}\left(-\frac{1}{2}x^{2}\left(a^{2}r_{e}^{-1}+r_{x}^{-1}\right)-x\left(dar_{e}^{-1}\right)\right)\right] = \exp\left[-\frac{1}{2}x^{2}\left(\widehat{a^{2}}r_{e}^{-1}+r_{x}^{-1}\right)-x\left(d\widehat{a}r_{e}^{-1}\right)\right].
$$
Z výrazů pro aposteriorno vidíme, že má stejný funkcionální tvar jako apriorno, tedy 
$$
\tilde{f}\left(a|d\right)=\mathcal{N}(\mu_{a},\sigma_{a}),\,\,\,\,\,\,\,\, \tilde{f}\left(x|d\right)=\mathcal{N}(\mu_{x},\sigma_{x})\propto\exp\left(-\frac{1}{2}\frac{(x-\mu_{x})^2}{\sigma_{x}}\right),
$$
kde tvarovací parametry $\mu_{a},\sigma_{a},\mu_{x},\sigma_{x}$ můžeme vyjádřit jako
$$
\sigma_{a}= \left(\widehat{x^{2}}r_{e}^{-1}+r_{a}^{-1}\right){}^{-1},\\
\mu_{a}= \sigma_{a}d\widehat{x}r_{e}^{-1},\\
\sigma_{x}= \left(\widehat{a^{2}}r_{e}^{-1}+r_{x}^{-1}\right){}^{-1},\\
\mu_{x}= \sigma_{x}d\widehat{a}r_{e}^{-1},
$$
přičemž momenty normálního rozdělení jsou $\widehat{a} = \mu_a$ a $\widehat{a^2} = \mu_a^2 + \sigma_a$.

Tím máme vyjádřené aposteriorno, které se postupně updatuje pro jednotlivé parametry dokola: $\widehat{a} \longrightarrow \widehat{x} \longrightarrow \widehat{a} \longrightarrow \dots$ Zapsáno jako algoritmus to vypadá následovně:

### VB algoritmus pro skalární dekompozici:
---
1. inicializace $d$, $r_e$, $r_a$ a $r_x$
2. zvolíme startovací $\sigma_a$ a $\mu_a$
3. iterujeme do dosažení konvergence
    - $\sigma_{x}$, $\mu_x$
    - $\sigma_{a}$, $\mu_a$
4. máme výsledné $\widehat{a}$ a $\widehat{x}$
---

Výsledek tedy vede na iterační algoritmus, u kterého máme zajišťěno, že konverguje k lokálnímu minimu. Tím pádem tedy samozřejmě záleží na tom, kde odstartujeme.

<img src="img_ot/l5_vb_scalar.png">

## Bayesovská lineární regrese

Též "Bayesian ridge regression", ukážeme si podobnost s Tichonovskou regularizací.

Připomeňme základní rovnici popisující lineární regresi jako
$$
\mathbf{y}=X\beta+\mathbf{e},
$$
kde obdobně jako nejmenší čtverce a další předpokládám rozdělení chyb jako $e_{i}=\mathcal{N}\left(0,\omega^{-1}\right),\ \forall i$. Z toho nám přirozeně plyne následující model dat
$$
f(\mathbf{y}|\beta,\omega)=\mathcal{N}\left(X\beta,\omega^{-1}I_{p}\right),
$$
s neznámými parametry $\beta$ a $\omega$. Protože se nyní díváme na problém bayesovským pohledem, každý neznámý parametr je pro nás náhodná proměnná, které je nutno přiřadit vhodné apriorní rozdělení. My se budeme držet v exponenciální rodině a apriorní rozdělení zvolíme následovně:
$$
f(\beta)= \mathcal{N}\left(\mathbf{0},I_{n}\right)=\underbrace{\left(2\pi\right)^{-\frac{n}{2}}}_{coef}\underbrace{\left|I_{n}\right|^{-\frac{1}{2}}}_{coef}\exp\left(-\frac{1}{2}\beta'I_{n}\beta\right),\\
f(\omega)= \mathcal{G}\left(c_{0},d_{0}\right)=\underbrace{\frac{c_{0}^{d_{0}}}{\Gamma(c_{0})}}_{coef}\omega^{c_{0}-1}\exp\left(-d_{0}\omega\right)
$$
Tím máme plně určený model a můžeme aplikovat Bayesovu variační metodu obdobně jako pro skalární případ.

Nejprve si vyjádříme logaritmus sdružené pravděpodobnosti jako
$$
\ln f(\mathbf{y},\beta,\omega) = \ln|\omega^{-1}I_{p}|^{-\frac{1}{2}} - \frac{1}{2}\omega(\mathbf{y}-X\beta)^T(\mathbf{y}-X\beta) - \frac{1}{2}\beta^T\beta + (c_{0}-1)\ln\omega-d_{0}\omega + const.
$$

Následně si vyádříme marginální pravděpodobnosti pro odhadované parametry $\beta$ a $\omega$ jako
$$
\tilde{f}(\beta|\mathbf{y}) \propto \exp\left(-\frac{1}{2}\widehat{\omega}(-2)\mathbf{y}^{T}X\beta-\frac{1}{2}\widehat{\omega}\beta^{T}X^{T}X\beta-\frac{1}{2}(\beta^{T}I_n\beta)\right)\\
\tilde{f}(\omega|\mathbf{y}) \propto \exp\left( (c_{0}+\frac{p}{2}-1)\ln\omega-(d_{0}+\frac{1}{2}(\mathbf{y}\mathbf{y}^{T}-2\mathbf{y}^{T}X\widehat{\beta}+\widehat{\beta^{T}X^{T}X\beta})\omega \right)
$$

Tvar marginál identifikujeme jako
$$
\tilde{f}(\beta|\mathbf{y})=\mathcal{N}(\mu_{\beta},\Sigma_{\beta})\\
\tilde{f}(\omega|\mathbf{y})=\mathcal{G}(c,d).
$$
s tvarovacími parametry $\mu_{\beta},\Sigma_{\beta},c,d$, které chceme vyjádřit. K tomu nám pomůže si rozepsat naše marginální pravděpodobnosti na členy logaritmické, lineární a kvadratické:
$$
\tilde{f}(\beta|\mathbf{y}) \propto \exp\left(-\frac{1}{2}(-2)\underbrace{\widehat{\omega}\mathbf{y}^{T}X}_{\text{linearni clen}}\beta-\frac{1}{2}\beta^{T}\underbrace{(\widehat{\omega}X^{T}X+I_n)}_{\text{kvadraticky clen}}\beta)\right)\\
\tilde{f}(\omega|\mathbf{y}) \propto \exp\left( \underbrace{(c_{0}+\frac{p}{2}-1)}_{\text{logaritmicky clen}}\ln\omega-\underbrace{(d_{0}+\frac{1}{2}(\mathbf{y}\mathbf{y}^{T}-2\mathbf{y}^{T}X\widehat{\beta}+\widehat{\beta^{T}X^{T}X\beta})}_{\text{linearni clen}}\omega \right).
$$

S využitím tohoto rozpisu tedy můžeme vyjádřit předpisy pro jednotlivé tvarovací parametry $\mu_{\beta},\Sigma_{\beta},c,d$ jako:
\begin{align}
\Sigma_{\beta}= & \left(\widehat{\omega}X^{T}X+I_{n}\right)^{-1}\\
\mu_{\beta}= & \Sigma_{\beta}\widehat{\omega}X^{T}\mathbf{y}=\left(\widehat{\omega}X^{T}X+I_{n}\right)^{-1}\widehat{\omega}X^{T}\mathbf{y}\\
c= & c_{0}+\frac{p}{2}\\
d= & d_{0}+\frac{1}{2}\left(\mathbf{y}\mathbf{y}^{T}-2\mathbf{y}^{T}X\widehat{\beta}+\widehat{\beta^{T}X^{T}X\beta}\right)
\end{align}
a příslušnými momenty:
$$
\widehat{\omega} = \frac{c}{d},\\
\widehat{\beta} = \mu_{\beta},\\
\widehat{\beta\beta^T} = \mu_{\beta}\mu_{\beta}^T + \Sigma_{\beta}.
$$

Tím máme sestaveny rovnice vstupující do iteračního algoritmu, kde je postupně vyčíslujeme. Protože je soustava implicitní, nezbývá nám než iterovat, pokud by jsme byli schopni soustavu zjednodušit na analytický tvar, iteracím se vyhneme.

### Příklad:

Rekapitulace použitých dat:

<img src="img_ot/l5_y_illcond.png"> | = | <img src="img_ot/l4_X_illcond.png"> | <img src="img_ot/l4_beta_true.png">
- | - | - | -
$\mathbf{y}$ | = |<p align="center"> $X$ |<p align="center"> $\beta_{true}$

Odhad pomocí Bayesovské regrese:

<img src="img_ot/l5_beta_VB.png"> 

<img src="img_ot/l5_beta_VB_omega.png"> 

## Rekapitulace výsledků:

|<img src="img_ot/l4_ols_illcond.png"> | <img src="img_ot/l4_tichonov.png"> |
| --- | --- |
| <img src="img_ot/l4_lasso.png"> | <img src="img_ot/l5_beta_VB.png"> | 



## Pár poznámek na závěr, co nás čeká dál:

- další modely vektoru $\beta$ - řídkost, hladkost
- neřešili jsme omezení hodnot vektoru $\beta$, např. pozitivitu
- vynechali jsme jakoukoliv diskusi o modelu měření $\mathbf{y}$
- neřešili jsme fyzikální podstatu problému
